### Ruben Abbou
## Tridiagonal System Solver
This project includes a data structure for square tridiagonal matrices and a solver for systems of tridiagonal equations.

In [1]:
from numpy import array, zeros

In [2]:
class Tridiag:
    '''
    Object type to represent tridiagonal matrices.
    Contains a multiplication and copying methods as well as a printing routine,
    which does not take up memory.
    '''
    def __init__(self, d, a, b):
        self.d = d
        self.a = a
        self.b = b
        self.n = len(d)
        assert len(self.a) == self.n-1
        assert len(self.b) == self.n-1

    def multiply(self, x):
        assert self.n == len(x)
        out = zeros(self.n)
        for i in range(self.n):
            if i-1 >= 0:
                out[i] += self.b[i-1]*x[i-1]
            if i+1 < self.n:
                out[i] += self.a[i]*x[i+1]
            out[i] += self.d[i]*x[i]
        return array(out)
    
    def copy(self):
        return Tridiag(array(self.d), array(self.a), array(self.b))
    
    def __str__(self):
        out = '['
        for i in range(self.n):
            if i != 0: out += ' '
            out += '['
            for j in range(self.n):
                if abs(i-j) > 1: out += '0   '
                elif j == i: out += "{:.2f}".format(self.d[i])
                elif i == j-1: out += "{:.2f}".format(self.a[i])
                elif j == i-1: out += "{:.2f}".format(self.b[j])
                else: print(i, j, abs(i-j))
                if j < self.n - 1: out += ' '
            out += ']'
            if i != self.n-1 : out += '\n'
        return out + ']'

In [3]:
def forward_elim(C, r):
    A = C.copy()
    x = array(r)
    for i in range(1, A.n):
        factor = A.b[i-1] / A.d[i-1]
        A.d[i] -= factor * A.a[i-1]
        A.b[i-1] = 0
        x[i] -= factor * x[i-1]
    return A, x

def back_substitution(C, r):
    x = zeros(C.n)
    x[C.n-1] = r[C.n-1] / C.d[C.n-1]

    for i in range(C.n-2, -1, -1):
        x[i] = (r[i] - C.a[i] * x[i+1]) / C.d[i]
    return x

def gauss_tridiag_solve(C, r):
    '''
    Gaussian elimination algorithm to solve Cx=r tridiagonal matrices
    Inputs:
        - C: Tridiag object
        - r: array
    Outputs:
        - y: solution vector
    '''
    if len(r) != C.n:
        raise NameError('dimensions do not match')
    if any(C.d == 0):
        raise NameError('cannot have zeros along the diagonal')
    
    C, r = forward_elim(C, r)
    x = back_substitution(C, r)
    return x

In [4]:
n = 5
d = array([4 + 0.1 * k for k in range(n)])
a = array([1 + 0.01 * k**2 for k in range(n-1)])
b = array([1 - 0.01 - 0.03 * k for k in range(n-1)])
C = Tridiag(d, a, b)
print(C)

[[4.00 1.00 0    0    0   ]
 [0.99 4.10 1.01 0    0   ]
 [0    0.96 4.20 1.04 0   ]
 [0    0    0.93 4.30 1.09]
 [0    0    0    0.90 4.40]]


In [5]:
y = array((1., 2., 3., 4., 5.))
r = C.multiply(y)

print(gauss_tridiag_solve(C, r) - y < 1e-12)

[ True  True  True  True  True]


As desired, $x=y$.

In [6]:
print(C)

[[4.00 1.00 0    0    0   ]
 [0.99 4.10 1.01 0    0   ]
 [0    0.96 4.20 1.04 0   ]
 [0    0    0.93 4.30 1.09]
 [0    0    0    0.90 4.40]]
